In [13]:
import numpy as np
import re
fuel_density_kgpl = 0.72 # kg/L for gasoline

def find_best_engine_edf(required_thrust_n, propulsion_type, rc_engines_summary_df, edf_summary_df):
    """Finds the best engine or EDF based on required thrust and propulsion type."""
    best_match = None
    min_thrust_diff = float('inf')

    if propulsion_type in ['fuel-powered', 'hybrid']:
        # Filter out engines with no estimated thrust
        engines_with_thrust = rc_engines_summary_df.dropna(subset=['Estimated Thrust (N)']).copy()
        for index, engine in engines_with_thrust.iterrows():
            thrust_diff = abs(engine['Estimated Thrust (N)'] - required_thrust_n)
            if thrust_diff < min_thrust_diff:
                min_thrust_diff = thrust_diff
                best_match = {'type': 'engine', 'component': engine.to_dict()}

    if propulsion_type in ['Electric', 'hybrid']:
        # Filter out EDFs with no thrust
        edfs_with_thrust = edf_summary_df.dropna(subset=['Thrust (N)']).copy()
        for index, edf in edfs_with_thrust.iterrows():
            thrust_diff = abs(edf['Thrust (N)'] - required_thrust_n)
            if thrust_diff < min_thrust_diff:
                min_thrust_diff = thrust_diff
                best_match = {'type': 'edf', 'component': edf.to_dict()}

    return best_match


def select_fuel_tanks(endurance_minutes, estimated_power_w, fuel_tank_df, engine_efficiency_decimal=0.20):
    """Selects the minimum number of fuel tanks to meet endurance requirements and calculates fuel volume and weight."""
    if estimated_power_w is None or estimated_power_w <= 0 or engine_efficiency_decimal is None or engine_efficiency_decimal <= 0:
        return [], 0, 0 # Cannot calculate fuel needed without power, efficiency or positive efficiency

    # Assuming average velocity and fuel energy density (example values)
    # Energy Density of Gasoline (J/L) - Example value
    energy_density_fuel_jpl = 34.2 * 10**6 # 34.2 MJ/L

    # Account for engine efficiency to get the required energy from fuel
    required_energy_from_fuel_j = (estimated_power_w * endurance_minutes * 60) / engine_efficiency_decimal

    # Calculate required fuel volume in Liters
    required_fuel_liters = required_energy_from_fuel_j / energy_density_fuel_jpl
    required_fuel_ml = required_fuel_liters * 1000.0
    required_fuel_kg = required_fuel_liters * fuel_density_kgpl


    # Sort fuel tanks by capacity ascending to select the smallest number
    fuel_tank_df_cleaned = fuel_tank_df.copy()
    # Assuming 'Capacity' is in 'ml' and is numeric
    # Need to convert 'Capacity' column to numeric
    def parse_capacity_ml(capacity_str):
        if isinstance(capacity_str, str):
            match = re.search(r'(\d+\.?\d*)\s*ml', capacity_str)
            if match:
                return float(match.group(1))
        return None

    fuel_tank_df_cleaned['Capacity_ml'] = fuel_tank_df_cleaned['Capacity'].apply(parse_capacity_ml)
    fuel_tank_df_cleaned = fuel_tank_df_cleaned.dropna(subset=['Capacity_ml']).sort_values(by='Capacity_ml', ascending=False)


    selected_tanks = []
    total_capacity_ml = 0

    # Select the largest tank which is smaller than 110% of remaining requirement
    while total_capacity_ml < required_fuel_ml:
        remaining_required_ml = required_fuel_ml - total_capacity_ml
        suitable_tanks = fuel_tank_df_cleaned[fuel_tank_df_cleaned['Capacity_ml'] < remaining_required_ml * 1.1]

        if suitable_tanks.empty:
             # If no tank is smaller than 110% of remaining, take the largest available tank
            if not fuel_tank_df_cleaned.empty:
                 selected_tanks.append(fuel_tank_df_cleaned.iloc[0].to_dict())
                 total_capacity_ml += fuel_tank_df_cleaned.iloc[0]['Capacity_ml']
            else:
                break # No tanks available
        else:
            # Select the largest tank from the suitable tanks
            largest_suitable_tank = suitable_tanks.iloc[0]
            selected_tanks.append(largest_suitable_tank.to_dict())
            total_capacity_ml += largest_suitable_tank['Capacity_ml']


    return selected_tanks, required_fuel_liters, required_fuel_kg


def select_batteries(sustained_power_w, endurance_minutes, battery_df_cleaned):
    """Selects the minimum number of batteries to meet power and endurance requirements."""
    if sustained_power_w is None or endurance_minutes is None or sustained_power_w <= 0 or endurance_minutes <= 0:
        return []

    # Required energy in Watt-hours
    required_energy_wh = (sustained_power_w * endurance_minutes) / 60

    # Calculate estimated Wh for each battery if not available
    # Assuming nominalV * capacity_Ah = Wh
    battery_df_cleaned['estimated_wh'] = battery_df_cleaned.apply(
        lambda row: row['nominalV'] * row['capacity_Ah'] if pd.notna(row['nominalV']) and pd.notna(row['capacity_Ah']) else None, axis=1
    )

    # Sort batteries by estimated_wh ascending to select the smallest number
    batteries_with_energy = battery_df_cleaned.dropna(subset=['estimated_wh']).sort_values(by='estimated_wh', ascending=False)

    selected_batteries = []
    total_energy_wh = 0

    # Greedily select batteries until required energy is met
    while total_energy_wh < required_energy_wh:
        remaining_required_wh = required_energy_wh - total_energy_wh
        suitable_batteries = batteries_with_energy[batteries_with_energy['estimated_wh'] < remaining_required_wh * 1.1]

        if suitable_batteries.empty:
            # If no battery is smaller than 110% of remaining, take the largest available battery
            if not batteries_with_energy.empty:
                selected_batteries.append(batteries_with_energy.iloc[0].to_dict())
                total_energy_wh += batteries_with_energy.iloc[0]['estimated_wh']
            else:
                break # No batteries available
        else:
            # Select the largest battery from the suitable batteries
            largest_suitable_battery = suitable_batteries.iloc[0]
            selected_batteries.append(largest_suitable_battery.to_dict())
            total_energy_wh += largest_suitable_battery['estimated_wh']


    return selected_batteries

**Reasoning**:
The helper functions are defined. Now, iterate through the optimized variants, use the helper functions to find and select the power components, and add a 'components: power' node to each variant.



In [14]:
import numpy as np
import re
fuel_density_kgpl = 0.72 # kg/L for gasoline

def find_best_engine_edf(required_thrust_n, propulsion_type, rc_engines_summary_df, edf_summary_df):
    """Finds the best engine or EDF based on required thrust and propulsion type."""
    best_match = None
    min_thrust_diff = float('inf')

    if propulsion_type in ['fuel-powered', 'hybrid']:
        # Filter out engines with no estimated thrust
        engines_with_thrust = rc_engines_summary_df.dropna(subset=['Estimated Thrust (N)']).copy()
        for index, engine in engines_with_thrust.iterrows():
            thrust_diff = abs(engine['Estimated Thrust (N)'] - required_thrust_n)
            if thrust_diff < min_thrust_diff:
                min_thrust_diff = thrust_diff
                best_match = {'type': 'engine', 'component': engine.to_dict()}

    if propulsion_type in ['Electric', 'hybrid']:
        # Filter out EDFs with no thrust
        edfs_with_thrust = edf_summary_df.dropna(subset=['Thrust (N)']).copy()
        for index, edf in edfs_with_thrust.iterrows():
            thrust_diff = abs(edf['Thrust (N)'] - required_thrust_n)
            if thrust_diff < min_thrust_diff:
                min_thrust_diff = thrust_diff
                best_match = {'type': 'edf', 'component': edf.to_dict()}

    return best_match


def select_fuel_tanks(endurance_minutes, estimated_power_hp, fuel_tank_df, cessna_fuel_consumption_rate_l_per_hp_hour=0.168):
    """Selects the minimum number of fuel tanks to meet endurance requirements based on Cessna 172 rate."""
    if estimated_power_hp is None or estimated_power_hp <= 0 or endurance_minutes is None or endurance_minutes <= 0:
        return [], 0, 0 # Cannot calculate fuel needed without power, endurance or positive values

    # Required fuel in liters for the given endurance and estimated power using Cessna 172 rate
    required_fuel_liters = estimated_power_hp * cessna_fuel_consumption_rate_l_per_hp_hour * (endurance_minutes / 60)
    required_fuel_ml = required_fuel_liters * 1000.0
    required_fuel_kg = required_fuel_liters * fuel_density_kgpl


    # Sort fuel tanks by capacity ascending to select the smallest number
    fuel_tank_df_cleaned = fuel_tank_df.copy()
    # Assuming 'Capacity' is in 'ml' and is numeric
    # Need to convert 'Capacity' column to numeric
    def parse_capacity_ml(capacity_str):
        if isinstance(capacity_str, str):
            match = re.search(r'(\d+\.?\d*)\s*ml', capacity_str)
            if match:
                return float(match.group(1))
        return None

    fuel_tank_df_cleaned['Capacity_ml'] = fuel_tank_df_cleaned['Capacity'].apply(parse_capacity_ml)
    fuel_tank_df_cleaned = fuel_tank_df_cleaned.dropna(subset=['Capacity_ml']).sort_values(by='Capacity_ml', ascending=False)

    # Impute missing weight using a linear relationship: 591 ml = 50g
    imputation_slope = 50 / 591 # grams per ml
    fuel_tank_df_cleaned['Weight_g'] = fuel_tank_df_cleaned.apply(
        lambda row: float(row['Weight'].replace(' g', '').replace(' kg', '')) if isinstance(row['Weight'], str) and ('g' in row['Weight'] or 'kg' in row['Weight']) else (row['Capacity_ml'] * imputation_slope if pd.notna(row['Capacity_ml']) else None) if pd.isna(row['Weight']) or (isinstance(row['Weight'], str) and row['Weight'].strip() == '-') else row['Weight'],
        axis=1
    )
    # Convert kg to grams if necessary (assuming weights without unit are in grams)
    fuel_tank_df_cleaned['Weight_g'] = fuel_tank_df_cleaned.apply(
        lambda row: row['Weight_g'] * 1000 if isinstance(row['Weight'], str) and 'kg' in row['Weight'] else row['Weight_g'],
        axis=1
    )


    selected_tanks = []
    total_capacity_ml = 0

    # Select the largest tank which is smaller than 110% of remaining requirement
    while total_capacity_ml < required_fuel_ml:
        remaining_required_ml = required_fuel_ml - total_capacity_ml
        suitable_tanks = fuel_tank_df_cleaned[fuel_tank_df_cleaned['Capacity_ml'] < remaining_required_ml * 1.1]

        if suitable_tanks.empty:
             # If no tank is smaller than 110% of remaining, take the largest available tank
            if not fuel_tank_df_cleaned.empty:
                 selected_tanks.append(fuel_tank_df_cleaned.iloc[0].to_dict())
                 total_capacity_ml += fuel_tank_df_cleaned.iloc[0]['Capacity_ml']
            else:
                break # No tanks available
        else:
            # Select the largest tank from the suitable tanks
            largest_suitable_tank = suitable_tanks.iloc[0]
            selected_tanks.append(largest_suitable_tank.to_dict())
            total_capacity_ml += largest_suitable_tank['Capacity_ml']


    return selected_tanks, required_fuel_liters, required_fuel_kg


def select_batteries(sustained_power_w, endurance_minutes, battery_df_cleaned):
    """Selects the minimum number of batteries to meet power and endurance requirements."""
    if sustained_power_w is None or endurance_minutes is None or sustained_power_w <= 0 or endurance_minutes <= 0:
        return []

    # Required energy in Watt-hours
    required_energy_wh = (sustained_power_w * endurance_minutes) / 60

    # Calculate estimated Wh for each battery if not available
    # Assuming nominalV * capacity_Ah = Wh
    battery_df_cleaned['estimated_wh'] = battery_df_cleaned.apply(
        lambda row: row['nominalV'] * row['capacity_Ah'] if pd.notna(row['nominalV']) and pd.notna(row['capacity_Ah']) else None, axis=1
    )

    # Sort batteries by estimated_wh ascending to select the smallest number
    batteries_with_energy = battery_df_cleaned.dropna(subset=['estimated_wh']).sort_values(by='estimated_wh', ascending=False)

    selected_batteries = []
    total_energy_wh = 0

    # Greedily select batteries until required energy is met
    while total_energy_wh < required_energy_wh:
        remaining_required_wh = required_energy_wh - total_energy_wh
        suitable_batteries = batteries_with_energy[batteries_with_energy['estimated_wh'] < remaining_required_wh * 1.1]

        if suitable_batteries.empty:
            # If no battery is smaller than 110% of remaining, take the largest available battery
            if not batteries_with_energy.empty:
                selected_batteries.append(batteries_with_energy.iloc[0].to_dict())
                total_energy_wh += batteries_with_energy.iloc[0]['estimated_wh']
            else:
                break # No batteries available
        else:
            # Select the largest battery from the suitable batteries
            largest_suitable_battery = suitable_batteries.iloc[0]
            selected_batteries.append(largest_suitable_battery.to_dict())
            total_energy_wh += largest_suitable_battery['estimated_wh']


    return selected_batteries

# Access the fuel tank dataframe
fuel_tank_df = dataframes['fuel tank'].copy()

# Define the target fuel consumption rate (Cessna 172)
cessna_fuel_consumption_rate_l_per_hp_hour = 0.168

for variant_obj in all_variants:
    required_thrust_n = float(variant_obj.engine_thrust.replace(' N', ''))
    propulsion_type = variant_obj.Propulsion_Types
    endurance_minutes = float(variant_obj.Endurance.replace(' minutes', ''))
    takeoff_weight_kg = float(variant_obj.Takeoff_Weight.replace(' kg', ''))
    payload_capacity_kg = float(variant_obj.Payload_Capacity.replace(' kg', ''))


    selected_engine = find_best_engine_edf(required_thrust_n, propulsion_type, rc_engines_summary_df, edf_summary_df)

    selected_fuel_tanks = []
    required_fuel_liters = 0
    required_fuel_kg = 0
    selected_batt = []

    if propulsion_type in ['fuel-powered', 'hybrid']:
        # Get estimated power in HP from the selected engine or fallback to thrust-based estimation
        estimated_power_hp = None
        if selected_engine and selected_engine['type'] == 'engine' and 'Power (HP)' in selected_engine['component'] and pd.notna(selected_engine['component']['Power (HP)']):
             estimated_power_hp = selected_engine['component']['Power (HP)']
        else:
             # Fallback if engine power in HP is not available, convert estimated power from thrust to HP (very rough)
             # Assuming average velocity to estimate power from thrust
             average_velocity_mps = 20 # Example: 72 km/h
             estimated_power_w = required_thrust_n * average_velocity_mps
             estimated_power_hp = estimated_power_w / 745.7 # Convert Watts to HP


        selected_fuel_tanks, required_fuel_liters, required_fuel_kg = select_fuel_tanks(endurance_minutes, estimated_power_hp, fuel_tank_df, cessna_fuel_consumption_rate_l_per_hp_hour)


    if propulsion_type in ['Electric', 'hybrid']:
        # Refine sustained power requirement estimation based on thrust and endurance
        # Assuming sustained power is proportional to thrust
        # This is a simplified model. A more accurate model would consider efficiency, speed, etc.
        # Assuming sustained power is roughly 50% of the power required for max thrust
        # Power (W) = Thrust (N) * Velocity (m/s)
        # Assuming average velocity (same as for fuel-powered)
        average_velocity_mps = 20 # Example: 72 km/h
        estimated_max_power_w = required_thrust_n * average_velocity_mps
        sustained_power_w = estimated_max_power_w * 0.5 # Assuming sustained power is 50% of max power
        if sustained_power_w <= 0:
             sustained_power_w = 1000 # Fallback to placeholder if estimation is not possible


        selected_batt = select_batteries(sustained_power_w, endurance_minutes, battery_df_cleaned)


    # Calculate total weight of selected components (excluding airframe and payload for now)
    total_components_weight = 0
    if selected_engine and selected_engine['component'] and 'Weight_kg' in selected_engine['component'] and pd.notna(selected_engine['component']['Weight_kg']):
        total_components_weight += selected_engine['component']['Weight_kg']

    if selected_fuel_tanks:
        # Sum the weight of selected fuel tanks, assume fuel weight is negligible for now
        # A more accurate model would calculate fuel weight based on volume and density
        for tank in selected_fuel_tanks:
             if 'Weight_g' in tank and pd.notna(tank['Weight_g']): # Use the new imputed weight column
                  total_components_weight += tank['Weight_g'] / 1000.0 # Convert grams to kg


    total_components_weight += required_fuel_kg # Add the weight of the fuel


    if selected_batt:
        for battery in selected_batt:
             if 'weight_g' in battery and pd.notna(battery['weight_g']):
                  total_components_weight += battery['weight_g'] / 1000.0 # Convert grams to kg


    # Estimate airframe weight
    # Assuming airframe weight is the takeoff weight minus the weight of other components and payload
    estimated_airframe_weight_kg = takeoff_weight_kg - total_components_weight - payload_capacity_kg


    # Add the selected components and estimated airframe to the variant object
    variant_obj.components = {
        'power': {
            'engine': selected_engine,
            'fuel_tanks': selected_fuel_tanks,
            'fuel': {'volume_liters': required_fuel_liters, 'weight_kg': required_fuel_kg},
            'batteries': selected_batt
        },
        'payload': {'weight_kg': payload_capacity_kg},
        'airframe': {'weight_kg': estimated_airframe_weight_kg}
    }

# Calculate MTOW for all variants first
for variant_obj in all_variants:
    selected_engine = variant_obj.components['power']['engine']
    selected_fuel_tanks = variant_obj.components['power']['fuel_tanks']
    required_fuel_kg = variant_obj.components['power']['fuel']['weight_kg']
    selected_batt = variant_obj.components['power']['batteries']
    payload_capacity_kg = variant_obj.components['payload']['weight_kg']
    estimated_airframe_weight_kg = variant_obj.components['airframe']['weight_kg']


    variant_obj.flight_profile = {}

    # Calculate Takeoff Weight (MTOW)
    variant_obj.flight_profile['mtow_kg'] = (
        (selected_engine['component']['Weight_kg'] if selected_engine and selected_engine['component'] and 'Weight_kg' in selected_engine['component'] and pd.notna(selected_engine['component']['Weight_kg']) else 0) +
        (sum([(tank['Weight_g'] if pd.notna(tank['Weight_g']) else 0) for tank in selected_fuel_tanks])/1000 if selected_fuel_tanks else 0) + # Sum the imputed weight in kg
        required_fuel_kg +
        (sum([batt['weight_g'] for batt in selected_batt])/1000 if selected_batt else 0) + # Convert battery weight to kg
        payload_capacity_kg +
        estimated_airframe_weight_kg
    )


# Now calculate other flight profile characteristics using the calculated MTOWs
min_mtow = min([v.flight_profile['mtow_kg'] for v in all_variants if 'mtow_kg' in v.flight_profile and pd.notna(v.flight_profile['mtow_kg'])]) if all_variants else 0
cessna_mtow_kg = 1150
cessna_altitude_m = 2000

for variant_obj in all_variants:
    required_thrust_n = float(variant_obj.engine_thrust.replace(' N', ''))
    propulsion_type = variant_obj.Propulsion_Types
    endurance_minutes = float(variant_obj.Endurance.replace(' minutes', ''))


    # Calculate Cruise Speed (simplified)
    # Assuming cruise power is 75% of estimated max power (required_thrust_n * average_velocity_mps)
    # And cruise speed is proportional to the cube root of the power ratio (for a simple drag model)
    # Cruise Power = 0.75 * Max Power
    # Cruise Speed = Max Velocity * (Cruise Power / Max Power)^(1/3)
    # Assuming Max Velocity is average_velocity_mps used for power estimation
    average_velocity_mps = 20 # Example: 72 km/h
    estimated_max_power_w = required_thrust_n * average_velocity_mps
    cruise_power_w = estimated_max_power_w * 0.75
    if estimated_max_power_w > 0:
         variant_obj.flight_profile['cruise_speed_mps'] = average_velocity_mps * (cruise_power_w / estimated_max_power_w)**(1/3)
    else:
         variant_obj.flight_profile['cruise_speed_mps'] = average_velocity_mps * (0.75)**(1/3) # Fallback if max power is zero


    # Calculate Rotate Speed (simplified)
    variant_obj.flight_profile['rotate_speed_mps'] = variant_obj.flight_profile['cruise_speed_mps'] * 0.5


    # Calculate Takeoff Distance (simplified formula)
    # A very basic formula: Distance = (MTOW * Rotate Speed^2) / (2 * g * (Thrust - Drag))
    # Assuming drag is negligible at takeoff speed and using max thrust
    # g is acceleration due to gravity (9.81 m/s^2)
    g = 9.81
    thrust_at_takeoff = required_thrust_n # Assuming max thrust at takeoff
    # Need to ensure thrust is greater than zero to avoid division by zero
    if thrust_at_takeoff > 0:
        variant_obj.flight_profile['takeoff_distance_m'] = (variant_obj.flight_profile['mtow_kg'] * variant_obj.flight_profile['rotate_speed_mps']**2) / (2 * g * thrust_at_takeoff)
    else:
        variant_obj.flight_profile['takeoff_distance_m'] = None # Cannot calculate takeoff distance if thrust is zero


    # Calculate Climb Rate (simplified)
    # Assuming climb rate is proportional to the difference between thrust and drag at climb speed
    # and inversely proportional to weight.
    # A very simplified approach: assume a constant climb angle (e.g., 10 degrees)
    # Climb Rate = Velocity * sin(Climb Angle)
    # Using rotate speed as a proxy for initial climb speed and a 10-degree angle
    climb_angle_rad = np.deg2rad(10) # 10 degrees
    variant_obj.flight_profile['climb_rate_mps'] = variant_obj.flight_profile['rotate_speed_mps'] * np.sin(climb_angle_rad)


    # Calculate Cruise Altitude (simplified)
    # Based on MTOW - a rough linear relationship
    # Assuming a base altitude of 100m for all UAVs
    base_altitude_m = 100
    # Assuming a linear increase of altitude with MTOW, calibrated roughly to Cessna 172
    # Cessna 172 MTOW is around 1150 kg, typical cruise altitude around 2000 m (6500 ft)
    # Let's assume a linear increase of (2000 - 100) m over (1150 - min_mtow) kg
    if variant_obj.flight_profile['mtow_kg'] is not None and min_mtow is not None and cessna_mtow_kg > min_mtow:
        altitude_increase_per_kg = (cessna_altitude_m - base_altitude_m) / (cessna_mtow_kg - min_mtow)
        estimated_altitude_m = base_altitude_m + (variant_obj.flight_profile['mtow_kg'] - min_mtow) * altitude_increase_per_kg
        # Cap the estimated altitude at 10% of the range in meters
        max_altitude_from_range_m = float(variant_obj.Range.replace(' km', '')) * 1000 * 0.1 if isinstance(variant_obj.Range, str) and 'km' in variant_obj.Range else float('inf')

        variant_obj.flight_profile['cruise_altitude_m'] = min(estimated_altitude_m, max_altitude_from_range_m)
    else:
        variant_obj.flight_profile['cruise_altitude_m'] = base_altitude_m # Default to base altitude if calculation is not possible


    # Calculate Climb Fuel/Energy (simplified)
    # Assuming climb requires a constant power output (related to climb rate and weight) for a certain duration
    # Duration of climb (minutes) = Cruise Altitude (m) / Climb Rate (m/min)
    # Climb Rate (m/min) = Climb Rate (m/s) * 60
    climb_rate_mpm = variant_obj.flight_profile['climb_rate_mps'] * 60
    if climb_rate_mpm > 0 and variant_obj.flight_profile['cruise_altitude_m'] is not None:
        climb_duration_minutes = variant_obj.flight_profile['cruise_altitude_m'] / climb_rate_mpm
    else:
        climb_duration_minutes = 5 # Default climb duration if climb rate is zero

    if propulsion_type in ['fuel-powered', 'hybrid']:
         # Using the Cessna 172 fuel consumption rate for climb as well (simplification)
         # Need to estimate climb power in HP
         # Assuming climb power is 1.2 times the estimated cruise power (placeholder)
         climb_power_hp = estimated_power_hp * 1.2

         variant_obj.flight_profile['climb_fuel_liters'] = climb_power_hp * cessna_fuel_consumption_rate_l_per_hp_hour * (climb_duration_minutes / 60)
         variant_obj.flight_profile['climb_fuel_kg'] = variant_obj.flight_profile['climb_fuel_liters'] * fuel_density_kgpl


    elif propulsion_type == 'Electric':
        # Assuming climb power is 1.2 times the cruise power (placeholder)
        climb_power_w = cruise_power_w * 1.2
        climb_energy_wh = (climb_power_w * climb_duration_minutes) / 60
        variant_obj.flight_profile['climb_energy_wh'] = climb_energy_wh


    # Calculate Cruise Distance and Fuel/Energy Consumption
    # Cruise Duration (minutes) = Endurance (minutes) - Climb Duration (minutes)
    cruise_duration_minutes = endurance_minutes - climb_duration_minutes
    if cruise_duration_minutes < 0:
         cruise_duration_minutes = 0 # Ensure cruise duration is not negative

    if propulsion_type in ['fuel-powered', 'hybrid']:
        # Fuel consumption during cruise using Cessna 172 rate
        # Need to use estimated cruise power in HP
        cruise_power_hp = estimated_power_hp * 0.75

        cruise_fuel_liters = cruise_power_hp * cessna_fuel_consumption_rate_l_per_hp_hour * (cruise_duration_minutes / 60)
        variant_obj.flight_profile['cruise_fuel_liters'] = cruise_fuel_liters
        variant_obj.flight_profile['cruise_fuel_kg'] = cruise_fuel_liters * fuel_density_kgpl


        # Cruise Distance
        variant_obj.flight_profile['cruise_distance_km'] = (variant_obj.flight_profile['cruise_speed_mps'] * cruise_duration_minutes * 60) / 1000 # Convert meters to km


    elif propulsion_type == 'Electric':
        # Energy consumption during cruise
        cruise_energy_wh = (cruise_power_w * cruise_duration_minutes) / 60
        variant_obj.flight_profile['cruise_energy_wh'] = cruise_energy_wh

        # Cruise Distance
        variant_obj.flight_profile['cruise_distance_km'] = (variant_obj.flight_profile['cruise_speed_mps'] * cruise_duration_minutes * 60) / 1000 # Convert meters to km

    # Calculate Total Range (sum of climb and cruise distance)
    # Assuming a short takeoff and landing distance that is negligible for total range calculation
    if 'cruise_distance_km' in variant_obj.flight_profile:
         variant_obj.flight_profile['total_range_km'] = variant_obj.flight_profile['cruise_distance_km']
    else:
         variant_obj.flight_profile['total_range_km'] = 0 # Default to 0 if cruise distance not calculated

## Inspect Components of Updated Variants

### Subtask:
Display the `components: power` information for each variant in a readable format.

In [15]:
for i, variant_obj in enumerate(all_variants[0:1]):
    print(f"--- Variant {i+1} ---")
    # Assuming the __dict__ representation is sufficient for inspection
    print_as_json(variant_obj.__dict__)
    print("-" * 20)

--- Variant 1 ---
{
  "score": 9.0,
  "wing_loading": "127.24 kg/m^2",
  "induced_drag_proportion_at_ld_max": "0.04 ",
  "thrust_to_weight_ratio": 0.3469710855760236,
  "lift_to_drag_ratio": "16.97 ",
  "engine_thrust": "623.88 N",
  "Takeoff_Weight": "183.29 kg",
  "Payload_Capacity": "13.11 kg",
  "Propulsion_Types": "fuel-powered",
  "Endurance": "322.55 minutes",
  "Range": "798.12 km",
  "Wingspan": "1.49 m",
  "power_budget": {
    "motor_kg": 4.99104,
    "battery_kg": 0,
    "fuel_kg": 16.7693745,
    "total_power_budget_kg": 21.760414500000003
  },
  "power_budget_total_kg": 21.760414500000003,
  "wing_area_sq_m": 2.510821917808219,
  "wing_loading_kg_per_sq_m": 73.0,
  "induced_drag_coefficient": 0.1124976688670135,
  "power_budget_to_mtow_ratio": 0.11872123138196303,
  "payload_to_mtow_ratio": 0.07152599705384909,
  "mtow_to_range_ratio": 0.22965218262917855,
  "components": {
    "power": {
      "engine": {
        "type": "engine",
        "component": {
          "ModelI